In [1]:
import undetected_chromedriver as uc
import json
from haralyzer import HarParser, HarPage
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from lxml import etree
from datetime import date
import calendar
import numpy as np

In [2]:
# address = '333 E Colorado Ave, Colorado Springs, CO 80903'
# address_den = '115 W 10th Ave, Denver, CO 80204'

In [23]:
def scrapeTrails(address):

    curr_date = date.today()
    today = str(calendar.day_name[curr_date.weekday()])
    doys = {'Sunday': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6}
    
    
    url = "https://9ioacg5nhe-dsn.algolia.net/1/indexes/alltrails_index3/query?x-algolia-agent=Algolia%20for%20JavaScript%20(4.8.6)%3B%20Browser"

    headers = CaseInsensitiveDict()
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"
    headers["Accept-Language"] = "en-US,en;q=0.9"
    headers["Accept-Encoding"] = "gzip, deflate, br"
    headers["Host"] = "9ioacg5nhe-dsn.algolia.net"
    headers["Origin"] = "https://www.alltrails.com"
    headers["User-Agent"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15"
    headers["Connection"] = "keep-alive"
    headers["Referer"] = "https://www.alltrails.com/"
    headers["Content-Length"] = "2686"
    headers["x-algolia-application-id"] = "9IOACG5NHE"
    headers["x-algolia-api-key"] = "63a3cf94e0042b9c67abf0892fc1d223"

    data = '{"query":"","hitsPerPage":1000,"analyticsTags":["auth:pro","platform:web","origin:explore","actor:user","lang:en","hastext:false"],"attributesToRetrieve":["ID","_geoloc","activities","area_id","area_name","area_slug","avg_rating","city_id","city_name","country_id","country_name","created_at","difficulty_rating","duration_minutes","duration_minutes_cycling","duration_minutes_hiking","duration_minutes_mountain_biking","duration_minutes_trail_running","elevation_gain","filters","has_profile_photo","is_closed","is_private_property","length","name","num_photos","num_reviews","photo_count","popularity","profile_photo_data","route_type","slug","state_id","state_name","type","units","user","verified_map_id","visitor_usage","area_name_en","city_name_en","country_name_en","state_name_en","name_en"],"attributesToHighlight":[],"filters":"((length>=0)) AND ((elevation_gain>=0)) AND (objectID:trail-10344041 OR objectID:trail-10002885 OR objectID:trail-10260451 OR objectID:trail-10351821 OR objectID:trail-10351817 OR objectID:trail-10294852 OR objectID:trail-10017018 OR objectID:trail-10035463 OR objectID:trail-10005393 OR objectID:trail-10031135 OR objectID:trail-10294792 OR objectID:trail-10239233 OR objectID:trail-10043013 OR objectID:trail-10028431 OR objectID:trail-10035470 OR objectID:trail-10009934 OR objectID:trail-10279945 OR objectID:trail-10235969 OR objectID:trail-10031144 OR objectID:trail-10280978 OR objectID:trail-10030644 OR objectID:trail-10016671 OR objectID:trail-10018317 OR objectID:trail-10111944 OR objectID:trail-10001851 OR objectID:trail-10340507 OR objectID:trail-10253874 OR objectID:trail-10273673 OR objectID:trail-10239674 OR objectID:trail-10299934 OR objectID:trail-10259768 OR objectID:trail-10010211 OR objectID:trail-10551515 OR objectID:trail-10026684 OR objectID:trail-10015640 OR objectID:trail-10017023 OR objectID:trail-10026711 OR objectID:trail-10041185 OR objectID:trail-10235681 OR objectID:trail-10111237 OR objectID:trail-10241649 OR objectID:trail-10258958 OR objectID:trail-10035010 OR objectID:trail-10277791 OR objectID:trail-10253281 OR objectID:trail-10295745 OR objectID:trail-10009207 OR objectID:trail-10307880 OR objectID:trail-10003461 OR objectID:trail-10035011 OR objectID:trail-10014975 OR objectID:trail-10235688 OR objectID:trail-10005478 OR objectID:trail-10241141 OR objectID:trail-10037015 OR objectID:trail-10000215 OR objectID:trail-10008249 OR objectID:trail-10269270 OR objectID:trail-10034964 OR objectID:trail-10746249 OR objectID:trail-10256323 OR objectID:trail-10034993 OR objectID:trail-10263807 OR objectID:trail-10031833 OR objectID:trail-10289334)","responseFields":["hits","hitsPerPage","nbHits"]}'


    resp = requests.post(url, headers=headers, data=data)

    print('Response Status: ' + str(resp.status_code))

    hits = json.loads(resp.content)
    #     with open('www.alltrails.com.har', 'r') as f:
#         har_parser = HarParser(json.loads(f.read()))

#     data = har_parser.har_data
#     # print(data.keys())



#     hits = json.loads(data['entries'][4]['response']['content']['text'])


    hikes = pd.DataFrame.from_dict(hits['hits'])

    # print(hikes)

    # for hit in hits['hits']:
    #     print(hit['name'])

    # print(hit.keys())

    hikes['duration_hours'] = hikes['duration_minutes']/60
    hikes['distance_miles'] = hikes['length']*.000621371
    
    #get hike url list
    hike_url = list('https://www.alltrails.com/'+hikes['slug'])
    hikes['hike_url'] = pd.DataFrame(hike_url)

    # hike_sample = hike_url[0:4]

    hike_dict = dict()
    weather = dict()

    # print(hike_url)

    driver = uc.Chrome()

    driver.get(hike_url[0])
    
    import time

    i = 20
    while i > -1:
        print('Solve Captcha in:', i, 'seconds', end='\r')
        time.sleep(1)
        i -= 1
    
    #solve captcha once

    #get metadata for each hike
    
    for hike in hike_url:
        attempts = 0
        success = False
        while attempts < 5 and not success:
            try:
                driver.get(hike)
                hike_soup = BeautifulSoup(driver.page_source)
                header = hike_soup.find('div', id='content')
                metadata = header.findChild('div')['data-react-props']
                metadata = json.loads(metadata)


                name = metadata['trail']['name']
                features = metadata['trailTags']['whatToSeeAndObstacles']
                last_review_date = metadata['reviews']['trail_reviews'][0]['date']
                last_review = metadata['reviews']['trail_reviews'][0]['comment']
                if metadata['weatherConditionsProps']['weatherForecast'] != None:
                    if doys[today] == 1:
                        weather_nar_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['narrative']
                        weather_nar_sun = 'n/a'
                        clouds_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['clds']
                        clouds_sun = 'n/a'
                        rain_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['pop']
                        rain_sun = 'n/a'
                        moon_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['lunar_phase']
                        moon_sun = 'n/a'

                    #weather_day = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['dow']
                    # weather_nar_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['narrative']
                    # weather_nar_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['day']['narrative']
                    # clouds_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['clds']
                    # clouds_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['day']['clds']
                    # rain_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['pop']
                    # rain_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['day']['pop']
                    # moon_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['lunar_phase']
                    # moon_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['lunar_phase']
                    
                    
                    else:
                        weather_nar_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['narrative']
                        weather_nar_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['day']['narrative']
                        clouds_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['clds']
                        clouds_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['day']['clds']
                        rain_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['day']['pop']
                        rain_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['day']['pop']
                        moon_sat = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][6 - doys[today]]['lunar_phase']
                        moon_sun = metadata['weatherConditionsProps']['weatherForecast']['forecasts'][7 - doys[today]]['lunar_phase']

                        #weather_nar_sat = weather_nar_sat + ' clouds: '+ str(clouds_sat)+ '%'
                    #weather_nar_sun = weather_nar_sun + ' clouds: '+ str(clouds_sun)+ '%'

                else:
                    #weather_day = 'n/a'
                    weather_nar_sat = 'n/a'
                    weather_nar_sun = 'n/a'
                    clouds_sat = 'n/a'
                    clouds_sun = 'n/a'
                    rain_sat = 'n/a'
                    rain_sun = 'n/a'
                    moon_sat = 'n/a'
                    moon_sun = 'n/a'

                    # weather_clouds = 'n/a'

                # weather[weather_day] = {'summary': weather_nar, 'clouds':weather_clouds}
                hike_info = {'features':features, 'last hiked': last_review_date, 'last review': last_review, 'saturday_weather': weather_nar_sat,'sunday_weather': weather_nar_sun, 'clouds_sat':clouds_sat,'clouds_sun': clouds_sun,'rain_sat':rain_sat,'rain_sun':rain_sun,'moon_phase_sat':moon_sat,'moon_phase_sun': moon_sun}
                hike_dict[name] = hike_info

                print('Processing:', name)
                # print(features)
                # print(hike_dict[name])
                success = True
            except:
                attempts += 1
                if attempts == 5:
                    pass

    # print(hike_dict)

    # driver.get('https://www.alltrails.com/trail/us/colorado/wheeler-lake')
    # test_hike = BeautifulSoup(driver.page_source)
    # test_header = test_hike.find('div', id='content')
    # test_metadata = test_header.findChild('div')['data-react-props']
    # test_metadata = json.loads(test_metadata)
    # test_name = metadata['trail']['name']
    # test_features = metadata['trailTags']['whatToSeeAndObstacles']
    # test_last_review_date = metadata['reviews']['trail_reviews'][0]['date']
    # test_last_review = metadata['reviews']['trail_reviews'][0]['comment']
    # test_metadata['weatherConditionsProps']['weatherForecast']['forecasts'][2]['lunar_phase']                                      

    # print(test_hike)

    metadata_df = pd.DataFrame.from_dict(hike_dict)
    metadata_df = pd.DataFrame.transpose(metadata_df)
    metadata_df.reset_index(inplace=True)

    # print(metadata_df)

    #get driving times
    maps_url = []
    n = 0
    for hike in hikes._geoloc:
        maps_url = maps_url + ['https://www.google.com/maps/dir/'+ address.replace(' ','+') + '/'  + str(hikes._geoloc[n]['lat']) + ',' + str(hikes._geoloc[n]['lng'])]
        n = n+1

    #get driving times from denver
    # maps_url_den = []
    # n = 0
    # for hike in hikes._geoloc:
    #     maps_url_den = maps_url_den + ['https://www.google.com/maps/dir/'+ address_den.replace(' ','+') + '/' + str(hikes._geoloc[n]['lat']) + ',' + str(hikes._geoloc[n]['lng'])]
    #     n = n+1

    # print(maps_url_den)

    # maps_url

    browser = webdriver.Chrome()

    times = []
    print('Getting driving times:')
    j = 0
    for dir in maps_url:
        browser.get(dir)
        maps = BeautifulSoup(browser.page_source, "html.parser")
        dom = etree.HTML(str(maps))
        if len(dom.xpath('//*[@id="section-directions-trip-0"]/div[1]/div[1]/div[1]/div[1]/span[1]'))>0:
            times = times + [dom.xpath('//*[@id="section-directions-trip-0"]/div[1]/div[1]/div[1]/div[1]/span[1]')[0].text]

        else: 
            times = times + ['n/a']
        j += 1
        print(str((round(j/len(maps_url) * 100, 1))) + ' % Complete')
        # print(times)
#     if add_den_address == 1:
#         times_den = []
    
#         for dir in maps_url_den:
#             browser.get(dir)
#             maps_den = BeautifulSoup(browser.page_source, "html.parser")
#             dom_den = etree.HTML(str(maps_den))
#             if len(dom_den.xpath('//*[@id="section-directions-trip-0"]/div[1]/div[1]/div[1]/div[1]/span[1]'))>0:
#                 times_den = times_den + [dom_den.xpath('//*[@id="section-directions-trip-0"]/div[1]/div[1]/div[1]/div[1]/span[1]')[0].text]

#             else: 
#                 times_den = times_den + ['n/a']
#             # print(times_den)
#     else:
#         pass
    # hike_times = []

    hike_times = pd.DataFrame()

    hike_times['names'] = hikes.name
    hike_times['driving_time_from_home'] = times

    # print(hike_times)

    total = []
    for time in hike_times.driving_time_from_home:
        if time != 'n/a':
            time = time.split(' ')

            try:
                hr = int(time[0])
                mins = int(time[2])
                total = total + [hr*60 + mins]
            except(IndexError):
                hr = int(time[0])
                mins = 0
                total = total + [hr*60 + mins]
        else:
            total = total + [999]
        #print(total)

    hike_times['time_mins'] = total

    hike_times = hike_times.sort_values('time_mins')

    # print(hike_times)

#     hike_times_den = []

#     hike_times_den = pd.DataFrame()

#     hike_times_den['names'] = hikes.name
#     hike_times_den['driving_time_from_denver'] = times_den

    # print(hike_times_den)
#     if add_den_address == 1:
#         total_den = []
#         for time in hike_times_den.driving_time_from_denver:
#             if time != 'n/a':
#                 time = time.split(' ')

#                 try:
#                     hr = int(time[0])
#                     mins = int(time[2])
#                     total_den = total_den + [hr*60 + mins]
#                 except(IndexError):
#                     hr = int(time[0])
#                     mins = 0
#                     total_den = total_den + [hr*60 + mins]
#             else:
#                 total_den = total_den + [999]
#             #print(total)
#     else:
#         pass

    # hike_times_den['time_mins'] = total_den

    # hike_times_den = hike_times_den.sort_values('time_mins')

    # print(hike_times_den)

    full_hike_table = hike_times.merge(metadata_df, left_on = 'names', right_on = 'index')

    # len(hike_times)

    metadata_df.to_csv('all_trails_data_sample.csv')

    # len(full_hike_table)

    hike_times.to_csv('driving_times_from_home.csv')

    # hike_times_den.to_csv('driving_times_from_Citzen.csv')

    full_hike_table = full_hike_table.drop(columns = ['index','time_mins'])

    full_hike_table.to_csv('hike_times_and_data.csv')

    # full_hike_table = full_hike_table.merge(hike_times_den)

    # full_hike_table = full_hike_table.drop(columns = ['time_mins'])

    all_hike_data = full_hike_table.merge(hikes, left_on = 'names', right_on = 'name')


    all_hike_data = all_hike_data.drop(columns = ['names','ID','state_id','length','slug','type', '_geoloc', 'route_type', 'visitor_usage','area_id','area_slug','city_id','country_id','verified_map_id','activities','profile_photo_data','has_profile_photo','num_photos','units','is_private_property','duration_minutes_trail_running','created_at','country_name','duration_minutes_mountain_biking','duration_minutes_hiking','duration_minutes','duration_minutes_cycling','duration_minutes_cycling','objectID'])

    all_hike_data.index = all_hike_data.name

    all_hike_data.insert(18, 'last review', all_hike_data.pop('last review'))

    all_hike_data['elevation_gain'] = all_hike_data['elevation_gain'] * 3.28084

    # all_hike_data = all_hike_data.drop(columns = ['name'])

    # all_hike_data.insert(1, 'driving_time_from_denver', all_hike_data.pop('driving_time_from_denver'))
    all_hike_data.insert(2, 'distance_miles', all_hike_data.pop('distance_miles'))
    all_hike_data.insert(9, 'last hiked', all_hike_data.pop('last hiked'))
    all_hike_data.insert(5, 'difficulty_rating', all_hike_data.pop('difficulty_rating'))
    all_hike_data.insert(3, 'duration_hours', all_hike_data.pop('duration_hours'))
    all_hike_data.insert(4, 'elevation_gain', all_hike_data.pop('elevation_gain'))
    all_hike_data.insert(10, 'clouds_sat', all_hike_data.pop('clouds_sat'))
    all_hike_data.insert(11, 'clouds_sun', all_hike_data.pop('clouds_sun'))


    all_hike_data.elevation_gain = round(all_hike_data.elevation_gain,3)
    all_hike_data.duration_hours = round(all_hike_data.duration_hours,3)
    all_hike_data.distance_miles = round(all_hike_data.distance_miles,2)

    all_hike_data = all_hike_data.rename(columns = {'elevation_gain':'elevation_gain_ft','rain_sun':'% chance rain sunday','rain_sat':'% chance rain saturday','clouds_sat':'% cloudy saturday','clouds_sun':'% cloudy sunday'})

    all_hike_data['is_day_hike'] = np.where(all_hike_data['duration_hours']< 10 , True, False)

    # all_hike_data['is_14er'] = np.where(all_hike_data['elevation_gain_ft']>=14000, True, False)

    all_hike_data 

    all_hike_data.to_csv('complete_hike_data'+str(date.today())+'.csv')

    return all_hike_data